# This notebook show how to do 
## 1 PCA
- PCA on MNIST handwritten dataset.
- Introduce some noise
- How to project data into new basis(principle components)
- Reconstruct data using only few principle components(Leave principle component capturing less variation in data). If noise is in later principle components direction then we have effectively denoised the images.

## 2  Fisher LDA

Let's start with PCA
 
We will load mnist data set and after loading the data set intentionally put some random noise at each pixel. As we will see later we can still recognise the digit. It means signal is still dominant and noise is not among the major direction(principle component) of variation in the digits. Hence if we remove later principle compoenent we can denoise the digits.

Please follow lecture note to understand operations in this notebook

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler


In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

**train Data is of shape (60000, 28, 28). It contains 60000 digits of shape 28X28. **
Data is loaded in **numpy  array object. It provides powerful multi dimensional array abstraction. 2d array provides matrix abstraction**. Checkout [numpy](http://www.numpy.org/) for quick numpy review.


In [ ]:
matplotlib.rcParams['figure.figsize'] = (1, 1)

In [ ]:
#digits = load_digits()
X = x_train

Y= y_train




In [ ]:
x_train.shape

Let's work only 10000 data points to avoid memory issue.

In [ ]:
X = x_train[0:10000,:]
Y = y_train[0:10000]
n_samples = X.shape[0]
print(n_samples)

Generally if your features are measured in different units(cm. km, light year etc) then standardizing them is a good practice.
Otherwise features measured in bigger units will dominate the variance.
Also search when to standardise the feature for more detail explanation.

**In this example we are not using StandardScaler. Each feature value is pixel intensity. Hence unit is not an issue here. I just kept it here to make sure you are aware of it. It is not the only way to standardize the data. To learn more read the sklearn documentation on standardizing data**

In [ ]:
#std_scale = StandardScaler().fit(X)
#X = std_scale.transform(X)

# Let's plot some digits

# Let set the seed so than we can reproduce the results across multiple run of notebook

In [ ]:
np.random.seed(1)

**Let's plot some random digits from the dataset by creating some random indexes**

In [ ]:
samples_to_plot = 10
indexes = np.random.randint(0, high=n_samples, size= [samples_to_plot])

In [ ]:
print(indexes)

See below how to plot images using plt.imshow. we need to reshape the data to 8x8 shape for visualization purpose.

In [ ]:
for idx in indexes:
    plt.imshow(X[idx,:],cmap= 'gray' )
    plt.title('digit = {}'.format(str(Y[idx])))
    plt.show()

**Adding some noise** Gaussian 

In [ ]:
X_noisy = X + 60*np.random.randn(X.shape[1], X.shape[2])

In [ ]:
for idx in indexes:
    plt.imshow(X_noisy[idx,:],cmap= 'gray' )
    plt.title('digit = {}'.format(str(Y[idx])))
    plt.show()

# We can stil lreconize the digits. so noise is not the primary direction of variation
# Let's do PCA to get rid of noise, assuming noise is not the primary direction  of variance and lies on lowest principle components direction

Let's first centralize the data

# vectorizing data

In [ ]:
X_noisy= X_noisy.reshape((X.shape[0], -1))
X_noisy.shape

In [ ]:
#std_scale = StandardScaler().fit(X_noisy)
#X_noisy = std_scale.transform(X_noisy)

In [ ]:
X_c = X_noisy - np.mean(X_noisy, axis=0)
print(X_c.shape)


In [ ]:
np.mean(X_noisy, axis=0).shape

AS per our discussion in the class we will use svd for doing PCA.

You can read about svd in detail [here](http://www.cs.cornell.edu/courses/cs3220/2010sp/notes/svd.pdf) or read your linear algerba book for SVD.

In [ ]:
# U, E, VT = svd(X)
U, E, VT = np.linalg.svd(X_c, full_matrices=False)

In [ ]:
U.shape, E.shape,VT.shape

# Let check if svd does matrix factorization or not

In [ ]:

X_c_recn = np.dot(U, np.diag(E))
X_c_recn.shape

In [ ]:
X_c_recn = np.dot(X_c_recn, VT)
X_c_recn.shape

In [ ]:
E[:20]

## Should get true on element wiser comparision

In [ ]:
np.allclose(X_c, X_c_recn)

Hence column of V or row of VT are eigen vectors of $X_c^TX_c$

# Let's plot the variance explained
This will help you to decide how many components to keep

In [ ]:
E_cumsum = np.cumsum(E)
print(E_cumsum)
total_variance = np.sum(E)


In [ ]:
plt.figure(figsize=(4,4))
plt.plot(E_cumsum/E_cumsum[-1])
plt.title('variance explained')
plt.xlabel('num components')
plt.ylabel('cumulative explained variance')
plt.show()

lets figure out 15% percentile

In [ ]:
index_per = int(len(E_cumsum)*.15)
print(index_per)

In [ ]:
plt.figure(figsize=(4,4))

plt.plot(index_per, E_cumsum[index_per]/E_cumsum[-1], 'ro')
plt.plot(E_cumsum/E_cumsum[-1])
plt.title('varaince explained')
plt.xlabel('num components')
plt.ylabel('cumulative explained variance')
plt.show()

looks  like using 117 component is also fine.

Let's reconstruct

In [ ]:
reduced_dim = index_per

In [ ]:

X_proj= np.dot(X_noisy, VT.T[:, :reduced_dim])

In [ ]:
print(X_proj.shape)

In [ ]:
X_reconstructed = np.dot(X_proj,VT[:reduced_dim, :])

In [ ]:
print(X_reconstructed.shape)

In [ ]:
for idx in indexes:
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize= (4,4))
    ax1.imshow(np.reshape(X_noisy[idx,:], (28,28)),cmap= 'gray' )
    ax1.set_title('noisy digit = {}'.format(str(Y[idx])))
    ax2.imshow(np.reshape(X_reconstructed[idx,:], (28,28)),cmap= 'gray' )
    ax2.set_title(' recon digit = {}'.format(str(Y[idx])))
    # Fine-tune figure; make subplots farther from each other.
    f.subplots_adjust(hspace=6.0, wspace = 6.0)
    
    plt.show()

# Let's visualize  first two component in project PCA space for all the data

In [ ]:
Y.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_proj[:, 0], X_proj[:, 1],
            c=Y, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('rainbow', 10))
plt.xlabel('principle component 1')
plt.ylabel('principle component 2')
plt.colorbar();

# Let's use sklearn

In [ ]:
from sklearn.decomposition import  PCA
from sklearn import preprocessing
#std_scale = preprocessing.StandardScaler().fit(X)
#X_train_std = std_scale.transform(X)
pca = PCA(n_components=reduced_dim, svd_solver='full')
pca.fit(X_noisy)

In [ ]:
X_pca= pca.fit_transform(X_noisy)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1],
            c=Y, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('rainbow', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();


# Doing PCA using covariance matrix(other method)

One can build covariance matrix directly and calculate its eigen vectors to find principle components

# Q1 Build covariance matrix and do eigen vector and value computation for doing PCA. Reconstruct data using only reduced_dim and plot the noisy and reconstructed images in indexes.

indexes contains some random integers build at the start of the notebook



See that eigenvalues are in ascending order

In [ ]:
# Write code block here

# Q2 Plot  first two components in the projected PCA space using scatter plot as done earlier.

It should match with svd  section plot

In [ ]:
# write code block here

## LDA part

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# working with full dataset

In [ ]:
X= x_train
Y= y_train

In [ ]:
from skimage.transform import  resize

Resizing 28x28 image to 8x8

<font color = 'red'> following operation will take some time. Don't run this cell again and again <font>

In [ ]:
new_img_size = (8,8)
X1 = np.zeros((X.shape[0],np.product(new_img_size) ))
for i in range(X.shape[0]):
    x = resize(X[i], new_img_size)
    X1[i,:] = x.reshape((-1))

In [ ]:
for idx in indexes:
    plt.imshow(np.reshape(X1[idx,:], new_img_size),cmap= 'gray' )
    plt.title('digit = {}'.format(str(Y[idx])))
    plt.show()

In [ ]:
X = X1

In [ ]:
lda = LDA(solver = 'eigen')
lda_fit = lda.fit(X, Y)


In [ ]:
X_lda = lda_fit.transform(X)

In [ ]:
X_lda.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_lda[:, 0], X_lda[:, 1],
            c=Y, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('rainbow', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
unique_class_label = np.unique(Y)

In [ ]:
print(unique_class_label)
num_classes = len(unique_class_label)


# Multiclass LDA see section 8.6.3.2 in the book

Let say we have $C$ classes

- $n_k=$ number of sample in class k
- $n = \sum_{k=1}^C n_k$ total sample

class $k$ mean $\mu_k= \frac{\sum_{i:y_i= k}x_i}{n_i}$ and over all mean $\mu= \frac{\sum_{i} x_i}{n}$

Objective function is $$\frac{|V^T S_B V|}{|V^TS_W V|}$$

where

- Within class scatter matrix  $S_w = \sum_{k=1}^C S_k$ and $S_k=  \sum_{i:y_i=k}((x_i - \mu_i)(x_i -\mu_i)^T)$
i.e class k unscaled covariance

- Between class scatter matrix $Sb =   \sum_{k=i}^{k=C} n_k (\mu_i - \mu)(\mu_i -\mu)^T$

Note maximum rank of $Sb$ is $C-1$


# computation of Sb and Sw

# Q3 Calculate Sb and Sw. Then solve for $S_b v = \lambda S_w v$ Generalized eigen value, vector

Hint 
- use from scipy import  linalg
- **Check exactly only nine eigen values are non zero. other are almost zero as we have 10 classes**

In [10]:
#Write code here

# Q4 Eigen vectors will not be not unit length. Normalize(make unit length) them

In [ ]:
# Write code block here

# Q 5.  Build projection matrix using 9 eigen vectors and project data into these 9 direction. Visualize first two components as done earlier using scatter plot.

Put eigen vectors along column. Make sure they are ordered by decreasing eigen value

In [ ]:
# Write code here